In [2]:
!nvidia-smi


Sun Jan 18 13:19:37 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
%%writefile task2.cu
#include <cuda_runtime.h>
#include <iostream>
#include <chrono>

#define N 1000000  // размер массивов

//
// CUDA-ядро для сложения массивов
//
__global__ void add_arrays(float* a, float* b, float* c) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // проверяем выход за границы
    if (idx < N) {
        c[idx] = a[idx] + b[idx];
    }
}

void run_test(int block_size) {
    float *h_a, *h_b, *h_c;
    float *d_a, *d_b, *d_c;

    // выделяем память на CPU
    h_a = new float[N];
    h_b = new float[N];
    h_c = new float[N];

    // инициализация массивов
    for (int i = 0; i < N; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    // выделяем память на GPU
    cudaMalloc(&d_a, N * sizeof(float));
    cudaMalloc(&d_b, N * sizeof(float));
    cudaMalloc(&d_c, N * sizeof(float));

    // копируем данные на GPU
    cudaMemcpy(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(float), cudaMemcpyHostToDevice);

    dim3 block(block_size);
    dim3 grid((N + block_size - 1) / block_size);

    // замер времени выполнения ядра
    auto start = std::chrono::high_resolution_clock::now();

    add_arrays<<<grid, block>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> time = end - start;

    std::cout << "Размер блока: " << block_size
              << " | Время: " << time.count() << " секунд\n";

    // очистка памяти
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    delete[] h_a;
    delete[] h_b;
    delete[] h_c;
}

int main() {
    std::cout << "Сложение массивов на GPU\n";

    // тестируем разные размеры блока
    run_test(128);
    run_test(256);
    run_test(512);

    return 0;
}


Overwriting task2.cu


In [12]:
!nvcc task2.cu -o task2
# Компиляция

In [13]:
# Запуск
!./task2


Сложение массивов на GPU
Размер блока: 128 | Время: 0.0110738 секунд
Размер блока: 256 | Время: 7.2121e-05 секунд
Размер блока: 512 | Время: 7.2375e-05 секунд
